In [0]:
-- This code creates or refreshes a streaming table named bronze_customers.
-- It ingests raw customer data from JSON files stored in the specified S3 path using Auto Loader (cloud_files).
-- The SELECT statement adds two columns: input_file_path (the source file path) and Ingestion_timestamp (the current timestamp).
-- The table is tagged with a 'bronze' quality property and includes a descriptive comment.
CREATE OR REFRESH STREAMING TABLE bronze_customers
  COMMENT 'Raw Customer Data Ingested into Bronze'
  TBLPROPERTIES ('quality' = 'bronze') AS
SELECT
  *,
  _metadata.file_path AS input_file_path,
  current_timestamp AS Ingestion_timestamp
FROM
  cloud_files(
    's3://dbxtrainingbucket/circuitbox/landing/operational_data/customers',
    'json',
    map('cloudFiles.inferColumnTypes', 'true')
  )

In [0]:
-- This code creates or refreshes a streaming table named silver_customers_clean.
-- It applies data quality constraints (expectations) to ensure valid customer records:
--   - customer_id must not be NULL (fails update if violated).
--   - customer_name must not be NULL (drops row if violated).
--   - telephone must be at least 10 characters.
--   - email must not be NULL.
--   - date_of_birth must be on or after 1920-01-01.
-- The table is tagged with a 'silver' quality property and includes a descriptive comment.
-- The SELECT statement reads from the bronze_customers streaming table, selects and casts relevant columns for further enrichment and cleaning.
CREATE OR REFRESH STREAMING TABLE silver_customers_clean(
  CONSTRAINT valid_customer_id EXPECT(customer_id IS NOT NULL) ON VIOLATION FAIL UPDATE,
  CONSTRAINT valid_customer_name EXPECT(customer_name IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_telephone EXPECT(LENGTH(telephone)>=10),                              
  CONSTRAINT valid_email EXPECT(email IS NOT NULL),
  CONSTRAINT valid_date_of_birth EXPECT(DATE_OF_BIRTH >='1920-01-01')  
)
  COMMENT 'Customer Data Cleaned and Enriched'
  TBLPROPERTIES ('quality' = 'silver') AS
SELECT
  customer_id,
  customer_name,
  CAST(date_of_birth AS DATE) AS date_of_birth,
  telephone,
  email,
  CAST(created_date AS DATE) AS created_date
FROM
  STREAM(LIVE.bronze_customers)

In [0]:
-- This code creates or refreshes a streaming table named silver_customers.
-- It is intended to store customer data with SCD Type 1 semantics, meaning updates overwrite existing records without retaining history.
-- The table is tagged with a 'silver' quality property and includes a descriptive comment.
CREATE OR REFRESH STREAMING TABLE silver_customers
COMMENT 'SCD Type 1 Customer Data'
TBLPROPERTIES ('quality' = 'silver')

In [0]:
-- SCD Type 1: Updates records in place without retaining historical versions.
-- When a change occurs, the existing record is overwritten with new values.
-- No history of previous values is maintained.
APPLY CHANGES INTO silver_customers
FROM STREAM(LIVE.silver_customers_clean)
KEYS(customer_id)
SEQUENCE BY created_date
STORED AS SCD TYPE 1